# Contents -->
## Cell 1. Fetching NVD Data from NVD API with missing components.
## Cell 2. Convert Json File to Excel File.
## Cell 3. Script which gives same result as 1st but it also includes missing component which is not found on NVD.
## cell 4. Script for converting json to excel, then read excel and match with main repository file and fetch component name and product name from NVD which are found then append in main repository other wise creat missing component file.


In [ ]:
# Fetching NVD Data from NVD API with missing components.

import requests
import pandas as pd
import time
import logging
import os
import time
from datetime import timedelta, datetime
# API_URL = "https://services.nvd.nist.gov/rest/json/cves/2.0"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

def generate_dynamic_url(keyword, severity):
    API_URL = f"https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch={keyword}&cvssV3Severity={severity}"
    print(f"* Generated API URL: {API_URL}")
    return API_URL

def search_nvd(keyword,severity,api_key):
    API_URL = generate_dynamic_url(keyword, severity)
  
    headers = {
        'Authorization': f'Bearer {api_key}'
    }

    params = {
        'resultsPerPage': 200,  
        'startIndex': 0
    }

    all_cves = []

    while True:
        print(f" * Making request to: {API_URL} with params: {params}")
        response = requests.get(API_URL, params=params, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            print(f"* API Response for {keyword} with severity {severity}: {data}")
        
            if 'vulnerabilities' in data:
                cves = data['vulnerabilities']
                print(f"* Found {len(cves)} CVEs for {keyword}")

                if len(cves) == 0:
                   print(f"** No CVEs found for {keyword} with severity {severity}")
                
                for cve in cves:
                    cve_id = cve['cve']['id']
                    descriptions = cve['cve']['descriptions']
                    description = next((d['value'] for d in descriptions if d['lang'] == 'en'), "No description available")
            
                    if 'cvssMetricV30' in cve['cve']['metrics']:
                        for metric in cve['cve']['metrics']['cvssMetricV30']:
                            cvss_v30 = metric.get('cvssData', {})
                            print(f"* CVSS 3.0 for {cve_id}: {cvss_v30}")
                            print(f"* Appending CVE {cve_id} for {severity} severity (v3.0).")
                            if cvss_v30:
                                cvss_v30_base_score = cvss_v30.get('baseScore', 0)
                                cvss_v30_severity = cvss_v30.get('baseSeverity', "")
                                all_cves.append({
                                                'Component Name': keyword,
                                                'CVE ID': cve_id,
                                                'Descriptions': description,
                                                'CVSS Version': "3.0",
                                                'Severity': cvss_v30_severity,
                                                'Base Score': cvss_v30_base_score
                                            })
                        
                    if 'cvssMetricV31' in cve['cve']['metrics']:
                        for metric in cve['cve']['metrics']['cvssMetricV31']:
                            cvss_v31 = metric.get('cvssData', {})
                            print(f"* CVSS 3.1 for {cve_id}: {cvss_v31}")
                            print(f"* Appending CVE {cve_id} for {severity} severity (v3.1).")
                            if cvss_v31:
                                cvss_v31_base_score = cvss_v31.get('baseScore', 0)
                                cvss_v31_severity = cvss_v31.get('baseSeverity', "")
                                all_cves.append({
                                            'Component Name': keyword,
                                            'CVE ID': cve_id,
                                            'Descriptions': description,
                                            'CVSS Version': "3.1",
                                            'Severity': cvss_v31_severity,
                                            'Base Score': cvss_v31_base_score
                                             })


                if len(cves) < 200:
                    print("Less than 200 CVEs found, breaking the loop.")
                    break
                params['startIndex'] += 200

            else:
                print(f"** No CVE data found for this {keyword}.")
                break

        elif response.status_code == 403:
            logger.warning("*** Received 403 error. Sleeping for 30 seconds and retrying...")
            time.sleep(30)
            continue  # Retrying
        else:
            logger.error(f"** Failed to retrieve data for {keyword}. HTTP Status Code: {response.status_code}")
            break

    return all_cves

def process_excel(input_file, severities ,api_key):
    ''' 
    Processing the Component file
    '''
    start_time = time.time()
    start_timestamp = datetime.now()
    print(f"Script started at: {start_timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
    print("---------------------------------------------")
    
    output_file_name = 'NVD_CVE_Data_WITH_3.0_AND_3.1.xlsx'
    
    if os.path.exists(output_file_name):
        os.remove(output_file_name)
        print(f"Deleted existing file: {output_file_name}")
        print("---------------------------------------------")
    df = pd.read_excel(input_file, engine='openpyxl')
    print(f"Excel data read successfully. Number of components: {len(df)}")
    print("---------------------------------------------")
   
    if 'Component Name' not in df.columns:
        print("'Component Name' column is missing in the input file.")
        print("---------------------------------------------")
        return pd.DataFrame()
    component_names = df['Component Name'].dropna().unique()
    all_cve_data = []

    total_components = len(component_names)

    for idx, component in enumerate(component_names):
        # # Break the loop if processed the 5 components
        # if idx >= max_components_to_process:
        #     print(f"Processed {max_components_to_process} components, stopping execution.")
        #     break
        print(f"* Processing component {idx + 1}/{total_components}: {component}")
        print("---------------------------------------------")
        # Fetch HIGH and CRITICAL CVE data
        for severity in severities:
            print(f"Fetching {severity} severity CVEs for component: {component}")
            print("---------------------------------------------")
            
            severity_cve_data = search_nvd(component, severity, api_key)
            if severity_cve_data:
                print(f"Found {len(severity_cve_data)} CVEs for severity {severity}.")
                print("---------------------------------------------")
                all_cve_data.extend(severity_cve_data)
            else:
                print(f"No CVEs found for severity {severity} for component {component}.")
                print("---------------------------------------------")
    
    if all_cve_data:
        cve_df = pd.DataFrame(all_cve_data)
        output_file_name = 'NVD_CVE_Data_WITH_3.0_AND_3.1.xlsx'
        cve_df.to_excel(output_file_name, index=False, engine='openpyxl')
        print(f"Data saved to {output_file_name}")
        print("---------------------------------------------")
        print("---------------------------------------------")
    else:
        print("No CVE data collected. Output file not created.")
        cve_df = pd.DataFrame()

    end_time = time.time()
    end_timestamp = datetime.now()
    total_time = timedelta(seconds=(end_time - start_time))

    print(f"Script ended at: {end_timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total processing time: {total_time}")
    print("------XXXX------XXXX---------XXXXXX-----")
    return cve_df
    
    

def main(input_file, api_key):
    '''
    calling function
    '''
    severity = ["HIGH","CRITICAL"]
    cve_df = process_excel(input_file, severity, api_key)
    return cve_df.head()


input_file = '/Users/f0tj5ln/code/CVE_Search/reference datas/component name.xlsx'  
api_key = "585bc7f5-b4b5-4e85-a9c3-40f21f9d7e98"

main(input_file, api_key)

In [ ]:
# Script for converting Json file to Excel file

import json
import pandas as pd
import os


with open('/Users/f0tj5ln/code/CVE_Search/reference datas/Components 3.json', 'r', encoding='utf-16') as file:
    data = json.load(file)
# Access the 'Type' attribute from each dictionary in the list
products = data[1].get('Products')
data_list = []
for product in products:
    module = product.get('module')
    file_name = product.get('info').get('filename')
    file_without_ext = file_name.replace(".dll", "")
    company_name = product.get('info').get('companyName')
    product_description = product.get('info').get('productDescription')
    product_version = product.get('info').get('productVersion')
    product_name = product.get('info').get('productName')

    data_list.append({
        'Module': module,
        'File Name': file_name,
        'File Without Extension': file_without_ext,
        'Company Name': company_name,
        'Product Name': product_name,
        'Product Description': product_description,
        'Product Version': product_version
        
    })

# Convert the list to a DataFrame
df = pd.DataFrame(data_list)

# Drop duplicate rows based on specific columns
df_unique = df.drop_duplicates(subset=['Module','File Name', 'File Without Extension', 'Product Description'])



# Save the DataFrame to an Excel file
output_file_path = '/Users/f0tj5ln/code/CVE_Search/extracted_json_data2.xlsx'

# Check if the file exists
if os.path.exists(output_file_path):
    # Delete the file
    os.remove(output_file_path)
    print(f"Previous file {output_file_path} deleted.")

# Save the new DataFrame to the same location
df_unique.to_excel(output_file_path, index=False, engine='openpyxl')
print(f"New data successfully saved to {output_file_path}")


In [ ]:
# Script which gives same result as 1st but it also includes missing component which is not found on NVD

import requests
import pandas as pd
import time
import logging
import os
import time
from datetime import timedelta, datetime
import shutil
import re
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry



# API_URL = "https://services.nvd.nist.gov/rest/json/cves/2.0"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

def generate_dynamic_url(keyword, severity):
    API_URL = f"https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch={keyword}&cvssV3Severity={severity}"
    print(f"* Generated API URL: {API_URL}")
    return API_URL

def search_nvd(keyword, severity, api_key):
    API_URL = generate_dynamic_url(keyword, severity)
  
    headers = {
        'Authorization': f'Bearer {api_key}'
    }

    params = {
        'resultsPerPage': 2000,  
        'startIndex': 0
    }

    all_cves = []

    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    while True:
        print(f" * Making request to: {API_URL} with params: {params}")
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = session.get(API_URL, params=params, headers=headers)
                break  # Exit the loop if successful
            except (requests.exceptions.ProxyError, requests.exceptions.ConnectionError, requests.exceptions.ChunkedEncodingError) as e:
                    print(f"*** Connection error encountered: {e}. Attempt {attempt + 1} of {max_retries}.")
                # Retry logic with a maximum of 3 attempts
            if attempt < max_retries - 1:
                print("Retrying...")
                time.sleep(30)  # Wait for 30 seconds before retrying
            else:
                print("Max retries reached. Moving to the next component.")
                time.sleep(30)
                continue  # Retry the same request after sleep
        
        if response.status_code == 200:
            data = response.json()
            print(f"* API Response for {keyword} with severity {severity}")
        
            if 'vulnerabilities' in data:
                cves = data['vulnerabilities']
                print(f"* Found {len(cves)} CVEs for {keyword}")

                if len(cves) == 0:
                   print(f"** No CVEs found for {keyword} with severity {severity}")
                
                for cve in cves:
                    cve_id = cve['cve']['id']
                    descriptions = cve['cve']['descriptions']
                    description = next((d['value'] for d in descriptions if d['lang'] == 'en'), "No description available")
                    if 'cvssMetricV30' in cve['cve']['metrics']:
                        for metric in cve['cve']['metrics']['cvssMetricV30']:
                            cvss_v30_source = metric.get('source', "")
                            cvss_v30_type = metric.get('type', "")
                            cvss_v30 = metric.get('cvssData', {})
                            if cvss_v30:
                                cvss_v30_base_score = cvss_v30.get('baseScore', 0)
                                cvss_v30_severity = cvss_v30.get('baseSeverity', "")
                                cvss_v30_vectorString = cvss_v30.get('vectorString', "")
                                all_cves.append({
                                                'Component Name': keyword,
                                                'CVE ID': cve_id,
                                                'Descriptions': description,
                                                'Type': cvss_v30_type,
                                                'Source': cvss_v30_source,
                                                'Vector String': cvss_v30_vectorString,
                                                'CVSS Version': "3.0",
                                                'Severity': cvss_v30_severity,
                                                'Base Score': cvss_v30_base_score
                                            })
                        
                    if 'cvssMetricV31' in cve['cve']['metrics']:
                        for metric in cve['cve']['metrics']['cvssMetricV31']:
                            cvss_v31_source = metric.get('source', "")
                            cvss_v31_type = metric.get('type', "")
                            cvss_v31 = metric.get('cvssData', {})
                            if cvss_v31:
                                cvss_v31_base_score = cvss_v31.get('baseScore', 0)
                                cvss_v31_severity = cvss_v31.get('baseSeverity', "")
                                cvss_v31_vectorString = cvss_v31.get('vectorString', "")
                                all_cves.append({
                                                'Component Name': keyword,
                                                'CVE ID': cve_id,
                                                'Descriptions': description,
                                                'Type': cvss_v31_type,
                                                'Source': cvss_v31_source,
                                                'Vector String': cvss_v31_vectorString,
                                                'CVSS Version': "3.1",
                                                'Severity': cvss_v31_severity,
                                                'Base Score': cvss_v31_base_score
                                             })


                if len(cves) < 200:
                    print("Less than 200 CVEs found, breaking the loop.")
                    break
                params['startIndex'] += 2000

            else:
                print(f"** No CVE data found for this {keyword}.")
                break

        elif response.status_code == 403:
            logger.warning("*** Received 403 error. Sleeping for 30 seconds and retrying...")
            time.sleep(30)
            continue  # Retry the same request after sleep
        else:
            logger.error(f"** Failed to retrieve data for {keyword}. HTTP Status Code: {response.status_code}")
            break

    return all_cves




def process_excel(input_file, NVD_file, Missing_component_file, severities ,api_key, archive_folder):
    ''' 
    Processing the Component file
    '''
    start_time = time.time()
    start_timestamp = datetime.now()
    print(f"Script started at: {start_timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
    print("---------------------------------------------")
    
    # output_file_name = [NVD_File, Missing_components_file]

    # for file_name in output_file_name:
    #     if os.path.exists(file_name):
    #         os.remove(file_name)
    #         print(f"deleted existing file: {file_name}")
    #         print("---------------------------------------------")
    #     else:
    #         print(f"File does not exist: {file_name}. Skipping deletion.")


    df = pd.read_excel(input_file, engine='openpyxl')
    print(f"Excel data read successfully. Number of components: {len(df)}")
    print("---------------------------------------------")
   
    if 'Component Name' not in df.columns:
        print("'Component Name' column is missing in the input file.")
        print("---------------------------------------------")
        return pd.DataFrame()
    component_names = df['Component Name'].dropna().unique()
    all_cve_data = []
    missing_components = []  # To store components not found on NVD

    # max_components_to_process = 5
    total_components = len(component_names)

    for idx, component in enumerate(component_names):
        # Break the loop if we have processed the first 10 components
        # if idx >= max_components_to_process:
        #     print(f"Processed {max_components_to_process} components, stopping further execution.")
        #     break
        component_found = False  # Flag to check if component data is found

        print(f"* Processing component {idx + 1}/{total_components}: {component}")
        print("---------------------------------------------")
        # Fetch HIGH and CRITICAL CVE data
        for severity in severities:
            print(f"Fetching {severity} severity CVEs for component: {component}")
            print("---------------------------------------------")
            
            severity_cve_data = search_nvd(component, severity, api_key)
            if severity_cve_data:
                print(f"Found {len(severity_cve_data)} CVEs for severity {severity}.")
                print("---------------------------------------------")
                all_cve_data.extend(severity_cve_data)
                component_found = True  # Mark as found if data is returned

        if not component_found:
            print(f"No CVEs found for component: {component}")
            missing_components.append({'Component Name': component})

    
    if all_cve_data:
        cve_df = pd.DataFrame(all_cve_data)
        output_file_name = 'new_sheet_NVD.xlsx'
        cve_df.to_excel(output_file_name, index=False, engine='openpyxl')
        print(f"Data saved to {output_file_name}")
        print("---------------------------------------------")
        print("---------------------------------------------")
    else:
        print("No CVE data collected. Output file not created.")
        cve_df = pd.DataFrame()

    # Save missing components to Excel
    if missing_components:
        missing_df = pd.DataFrame(missing_components)
        missing_output_file = 'Missing_Components.xlsx'
        missing_df.to_excel(missing_output_file, index=False, engine='openpyxl')
        print(f"Missing components saved to {output_file_not_found}")
    else:
        print("No missing components found. Missing components file not created.")
    print("---------------------------------------------")


    end_time = time.time()
    end_timestamp = datetime.now()
    total_time = timedelta(seconds=(end_time - start_time))

    print(f"Script ended at: {end_timestamp.strftime('%Y-%m-%d %H:%M:%S')}")    
    print(f"Total processing time: {total_time}")
    print("------XXXX------XXXX---------XXXXXX-----")

    return cve_df, missing_df


def main(input_file, NVD_file, Missing_component_file, api_key):
    '''
    calling function
    '''
    severities = ["HIGH", "CRITICAL"]
    cve_df, missing_component_df = process_excel(input_file, NVD_file, Missing_component_file, severities, api_key)
    return cve_df.head(), missing_component_df.head()


input_file = 'Gives Input file path where it reads the data'  
NVD_file = 'Gives the output file path if exist'
Missing_component_file = 'Give missing component file path if exist'
api_key = "585bc7f5-b4b5-4e85-a9c3-40f21f9d7e98" 

main(input_file,NVD_file, Missing_component_file, api_key)

In [ ]:
# Script for converting json to excel, then read excel and match with main repository file and fetch component name and product name from NVD which are found then append 
# in main repository other wise creat missing component file.
import json
import requests
import pandas as pd
import time
import logging
import os
import time
from datetime import timedelta, datetime
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import warnings
warnings.filterwarnings("ignore")

yellow = "\033[33m"
reset = "\033[0m"
green = "\033[32m"
red = "\033[31m"
cyan = "\033[36m"


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()


def json_to_excel(json_path:str)->pd.DataFrame:
    '''
    Convert Json File to Excel file.
    '''
    with open(json_path, 'r', encoding='utf-16') as file:
        data = json.load(file)

    # Access the 'Type' attribute from each dictionary in the list
    products = data[1].get('Products')
    data_list = []
    for product in products:
        module = product.get('module')
        file_name = product.get('info').get('filename')
        # file_without_ext = file_name.replace(".dll", "")
        company_name = product.get('info').get('companyName')
        product_description = product.get('info').get('productDescription')
        product_version = product.get('info').get('productVersion')
        product_name = product.get('info').get('productName')

        data_list.append({
                            'Module': module,
                            'File Name': file_name,
                            'Product Name': product_name,
                            'Product Description': product_description,
                            'Company Name': company_name,
                            'Product Version': product_version    
                        })
    df = pd.DataFrame(data_list)
    df_unique = df.drop_duplicates(subset=['Module','File Name', 'Product Description'])
    return df_unique

def merge_product_component(file1: pd.DataFrame, file2: str) -> pd.DataFrame:
    df2 = pd.read_excel(file2, engine='openpyxl') 
    file_to_component = df2.set_index('File Name')['Component Name'].to_dict()
    file1['Component Name'] = file1['File Name'].map(file_to_component)
    output_columns_file1 = ['Module', 'File Name', 'Company Name', 'Product Name', 'Component Name', 'Product Description',  'Product Version']
    output_columns_df2 = ['File Name', 'Component Name']
    return file1[output_columns_file1], df2[output_columns_df2]


def generate_dynamic_url(keyword, severity):
    API_URL = f"https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch={keyword}&cvssV3Severity={severity}"
    return API_URL


def search_nvd(keyword, severity, api_key):
    API_URL = generate_dynamic_url(keyword, severity)
  
    headers = {
        'Authorization': f'Bearer {api_key}'
    }

    params = {
        'resultsPerPage': 2000,  
        'startIndex': 0
    }

    all_cves = []

    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    while True:
        print(f" * Making request to: {API_URL} with params: {params}")
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = session.get(API_URL, params=params, headers=headers)
                break  # Exit the loop if successful
            except (requests.exceptions.ProxyError, requests.exceptions.ConnectionError, requests.exceptions.ChunkedEncodingError) as e:
                    print(f"*** Connection error encountered: {e}. Attempt {attempt + 1} of {max_retries}.")
                # Retry logic with a maximum of 3 attempts
            if attempt < max_retries - 1:
                print("Retrying...")
                time.sleep(30)  # Wait for 30 seconds before retrying
            else:
                print("Max retries reached. Moving to the next component.")
                time.sleep(30)
                continue  # Retry the same request after sleep
        
        if response.status_code == 200:
            data = response.json()
            print(f"* API Response for {keyword} with severity {severity}")
        
            if 'vulnerabilities' in data:
                cves = data['vulnerabilities']
                print(f"* Found {len(cves)} CVEs for {keyword}")

                if len(cves) == 0:
                   print(f"{red}** No CVEs found for {keyword} with severity {severity}{reset}")
                
                for cve in cves:
                    cve_id = cve['cve']['id']
                    descriptions = cve['cve']['descriptions']
                    description = next((d['value'] for d in descriptions if d['lang'] == 'en'), "No description available")
                    if 'cvssMetricV30' in cve['cve']['metrics']:
                        for metric in cve['cve']['metrics']['cvssMetricV30']:
                            cvss_v30_source = metric.get('source', "")
                            cvss_v30_type = metric.get('type', "")
                            cvss_v30 = metric.get('cvssData', {})
                            if cvss_v30:
                                cvss_v30_base_score = cvss_v30.get('baseScore', 0)
                                cvss_v30_severity = cvss_v30.get('baseSeverity', "")
                                cvss_v30_vectorString = cvss_v30.get('vectorString', "")
                                all_cves.append({
                                                'Component Name': keyword,
                                                'CVE ID': cve_id,
                                                'Descriptions': description,
                                                'Type': cvss_v30_type,
                                                'Source': cvss_v30_source,
                                                'Vector String': cvss_v30_vectorString,
                                                'CVSS Version': "3.0",
                                                'Severity': cvss_v30_severity,
                                                'Base Score': cvss_v30_base_score
                                            })
                            
                        
                    if 'cvssMetricV31' in cve['cve']['metrics']:
                        for metric in cve['cve']['metrics']['cvssMetricV31']:
                            cvss_v31_source = metric.get('source', "")
                            cvss_v31_type = metric.get('type', "")
                            cvss_v31 = metric.get('cvssData', {})
                            if cvss_v31:
                                cvss_v31_base_score = cvss_v31.get('baseScore', 0)
                                cvss_v31_severity = cvss_v31.get('baseSeverity', "")
                                cvss_v31_vectorString = cvss_v31.get('vectorString', "")
                                all_cves.append({
                                                'Component Name': keyword,
                                                'CVE ID': cve_id,
                                                'Descriptions': description,
                                                'Type': cvss_v31_type,
                                                'Source': cvss_v31_source,
                                                'Vector String': cvss_v31_vectorString,
                                                'CVSS Version': "3.1",
                                                'Severity': cvss_v31_severity,
                                                'Base Score': cvss_v31_base_score
                                             })
                    


                if len(cves) < 200:
                    print("Less than 200 CVEs found, breaking the loop.")
                    break
                params['startIndex'] += 2000

            else:
                print(f"{red}** No CVE data found for this {keyword}{reset}.")
                break

        elif response.status_code == 403:
            logger.warning(f"{red}*** Received 403 error. Sleeping for 30 seconds and retrying...{reset}")
            time.sleep(30)
            continue  # Retry the same request after sleep
        else:
            logger.error(f"{red}** Failed to retrieve data for {keyword}. HTTP Status Code: {response.status_code}{reset}")
            break

    return all_cves

def process_excel(json_file, repo_file, get_components_file, missing_components_file, severities, api_key):
    ''' 
    Processing the Component file
    '''
    start_time = time.time()
    start_timestamp = datetime.now()
    print(f"Script started at: {start_timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
    print("---------------------------------------------")

    product_df = json_to_excel(json_file)
    print("*** Step 1st Json to Excel Started")

    df,df2 = merge_product_component(product_df,repo_file)
    print("*** Step 2 product and component dataframe merged")
    
    if os.path.exists(get_components_file):
        os.remove(get_components_file)
        print(f"deleted existing file: {get_components_file}")
        print("---------------------------------------------")
    else:
        print(f"File does not exist: {get_components_file}. Skipping deletion.")
    
    if os.path.exists(missing_components_file):
        os.remove(missing_components_file)
        print(f"deleted existing file: {missing_components_file}")
        print("---------------------------------------------")
    else:
        print(f"File does not exist: {missing_components_file}. Skipping deletion.")


    print(f"Excel data read successfully. Number of components: {len(df)}")
    print("---------------------------------------------")
   
    if 'Product Name' not in df.columns or 'Component Name' not in df.columns:
        print("Required columns ('Product Name' or 'Component Name') are missing in the input file.")
        print("---------------------------------------------")
        return pd.DataFrame()
    
    df['Query Name'] = df['Component Name'].fillna(df['Product Name']).replace("®", "").replace("  ", " ")  # Use Component Name if available, else Product Name
    unique_queries = df['Query Name'].drop_duplicates().tolist()
    print(f"Number of unique queries to fetch: {len(unique_queries)}")

    all_cve_data = []
    missing_components = []
    processed_queries = set()  # To track already processed queries
    
    max_components_to_process = 30
    total_components = len(unique_queries)
    print(total_components)
    for idx, query_name in enumerate(unique_queries):
        # # Break the loop if we have processed the first 5 components
        if idx >= max_components_to_process:
            print(f"Processed {max_components_to_process} components, stopping further execution.")
            break
        if query_name in processed_queries:
            continue  # Skip already processed queries
        component_found = False
        print(f"* Processing query {idx + 1}/{max_components_to_process}: {cyan}{query_name}{reset}")
        print("---------------------------------------------")

       
        # Fetch HIGH and CRITICAL CVE data
        for severity in severities:
            print(f"{yellow}Fetching {severity} severity CVEs for component --> {query_name}{reset}")
            print("---------------------------------------------")
            
            severity_cve_data = search_nvd(query_name, severity, api_key)
            if severity_cve_data:
                print(f"{green}Found {len(severity_cve_data)} CVE IDs for severity {severity}.{reset}")
                print("---------------------------------------------")
                all_cve_data.extend(severity_cve_data)
                component_found = True  # Mark as found if data is returned
            
        if component_found:
        # Append query_name and corresponding file names to file2 if not already present
            matching_rows = df[df['Query Name'] == query_name]

            for _, row in matching_rows.iterrows():
                file_name = row['File Name']
                component_name = row['Query Name']

                # Check if file name already exists in file2
                if file_name not in df2['File Name'].values:
                    # Append new row to file2
                    new_row = pd.DataFrame({'File Name': [file_name], 'Component Name': [component_name]})
                    df2 = pd.concat([df2, new_row], ignore_index=True)
    
    
        else:
            # Add all rows corresponding to the missing query
            matching_rows = df[df['Query Name'] == query_name].to_dict('records')
            missing_components.extend(matching_rows)
            print(f"{query_name} not found in NVD database.")
            print("---------------------------------------------")

        processed_queries.add(query_name)

    df2.drop_duplicates(subset=['File Name'], inplace=True)  # Ensure file names are unique
    df2.to_excel(repo_file, index=False, engine='openpyxl')  # Overwrite the existing file2
    print(f"{green}Updated file2 saved to {repo_file}{reset}")
    print("---------------------------------------------")
    
    if all_cve_data:
        cve_df = pd.DataFrame(all_cve_data)
        output_file_name = get_components_file
        cve_df.to_excel(output_file_name, index=False, engine='openpyxl')
        print(f"{green}Data saved to {output_file_name}{reset}")
        print("---------------------------------------------")
        print("---------------------------------------------")
    else:
        print("No CVE data collected. Output file not created.")
        cve_df = pd.DataFrame()

    # Save missing components to Excel
    if missing_components:
        missing_df = pd.DataFrame(missing_components).drop_duplicates(subset=['Module', 'File Name'])
        missing_output_file = missing_components_file
        missing_df.to_excel(missing_output_file, index=False, engine='openpyxl')
        print(f"{red}Missing components saved to {missing_output_file}{reset}")
    else:
        print("No missing components found. Missing components file not created.")
    print("---------------------------------------------")

    end_time = time.time()
    end_timestamp = datetime.now()
    total_time = timedelta(seconds=(end_time - start_time))

    print(f"Script ended at: {end_timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total processing time: {total_time}")
    print("------XXXX------XXXX---------XXXXXX-----")
    return cve_df, missing_df
    
    

def main(json_file, repo_file, api_key, get_components_file, missing_components_file):
    '''
    calling function
    '''
    severities = ["HIGH", "CRITICAL"]
    cve_df, missing_df = process_excel(json_file, repo_file, get_components_file, missing_components_file, severities, api_key)
    return cve_df.head(), missing_df.head()


json_file = '/Users/f0tj5ln/code/CVE_Search/reference datas/Components 3.json'
repo_file = "/Users/f0tj5ln/code/CVE_Search/reference datas/Export copy.xlsx"
get_components_file = '/Users/f0tj5ln/code/CVE_Search/Product_data_from_Json2.xlsx'
missing_components_file = '/Users/f0tj5ln/code/CVE_Search/Missing_Components_Product_data_from_Json2.xlsx'  
api_key = "585bc7f5-b4b5-4e85-a9c3-40f21f9d7e98"

main(json_file, repo_file, api_key, get_components_file, missing_components_file)
